# Automatizar o download das planilhas geradas pelo site da SSP-SP
http://www.ssp.sp.gov.br/transparenciassp/

Para esta tarefa será utilizado o Selenium para automatizar a navegação pelo site até a exportação das planilhas de:
- Roubos de Celulares
- Roubos de Veículos
- Furtos de Veículos

## Importar as bibliotecas necessárias para a coleta dos dados

In [ ]:
#importando bibliotecas
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from time import localtime, strftime
from selenium.webdriver.chrome.options import Options



# Definir campos e periodo que serão baixados

Serão baixados os dados de:
   - Furto de Veículos
   - Roubo de Veículos
   - Roubo de Celular
    
No periodo de 2010 à 2020

In [ ]:
meses = range(12,0,-1)
anos = range(20,9,-1)
campos = ['cphBody_btnFurtoVeiculo', 'cphBody_btnRouboVeiculo', 'cphBody_btnRouboCelular']


# Definir local de download e webdriver

Os arquivos estão sendo salvos em uma pasta chamada __*data*__ dentro do diretório onde o script esta sendo executado

- Para usar o Firefox em modo headless utilize _**"1"**_
- Para usar o Chrome utilize **_"2"_**

In [ ]:
#Localiza o diretório onde o script está sendo executado e acrescenta no caminho o diretório escolhido para salvar
path_download = os.getcwd() + '\\data'


In [ ]:
#Seleção de Webdriver
print('Digite 1 para Firefox, 2 para Chrome\t')
x = input()

if x=='1':
    browser = 0
    print('Selecionado 1-Firefox')
elif x=='2':
    browser = 1
    print('Selecionado 2-Chrome')
else:
    print('Valor de entrada não é valido, selecionado 2-Chrome')
    browser = 1

    
    
if browser == 0:
    fp = webdriver.FirefoxProfile()
    fp.set_preference("browser.download.folderList",2)
    fp.set_preference("browser.download.manager.showWhenStarting",False)
    fp.set_preference("browser.download.dir", path_download)
    fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")
    fo = webdriver.FirefoxOptions()
    fo.headless = True

    driver = webdriver.Firefox(firefox_profile=fp, options=fo)
    print('Webdriver do Firefox configurado e inciado')
else:
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    #chrome_options.add_argument("--no-sandbox") # linux only
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--verbose")

    desired_caps = {'prefs': {'download': {'default_directory': path_download, "directory_upgrade": True}}}
    driver = webdriver.Chrome(options=chrome_options, desired_capabilities = desired_caps)
    driver.set_page_load_timeout(10800)
    print('Webdriver do Chrome configurado e iniciado')    

# Coleta de dados

Navegação pelo site e exportação dos arquivos escolhidos

In [ ]:
# Carrega o site para coleta
driver.get('http://www.ssp.sp.gov.br/transparenciassp/')
sleep(10)

for caminho in campos:
    #Localizar o campo  escolhido, aguardar o carregamento da página e clicar no botão encontrado
    print(strftime("Coletando dados de: "+str(caminho[11:]) + ": %d %b %Y %H:%M:%S", localtime()))
    wait = WebDriverWait(driver, 10800) #Determina uma espera de 10800 segundos
    
    try:
        sleep(5)
        # Aguarda a remoção do "blockUI"
        wait.until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='blockUI blockOverlay']")))
        
    finally:
        # Localiza o elemento (Roubo Veiculo / Furto Veiculo / Roubo Celular) e clica
        elem = driver.find_element_by_id(caminho)
        elem.click()
        sleep(5)

        # Loop for para seleção dos anos
        for i in anos:
            xpath = ("//*[@id='cphBody_lkAno{}']".format(i))
            wait = WebDriverWait(driver, 10800)
            
            try:
                # Aguarda a remoção do "blockUI"
                wait.until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='blockUI blockOverlay']")))
                # Aguarda o elemento ser clicável
                wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
                
            finally:
                # Localiza o elemento ano e clica
                elemano = driver.find_element_by_xpath(xpath)
                elemano.click()
                print('Clicado em ano: ', i)
                sleep(5)

                # Loop for para seleção dos meses
                for j in meses:
                    
                    try:
                        # Aguarda a remoção do "blockUI"
                        wait.until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='blockUI blockOverlay']")))
                        # Aguarda o elemento ser clicável
                        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cphBody_lkMes{}"]'.format(j))))
                        
                    finally:
                        # Localiza o elemento mês e clica
                        elemmes = driver.find_element_by_xpath('//*[@id="cphBody_lkMes{}"]'.format(j))
                        elemmes.click()
                        print('Clicado em mes: ', j)
                        sleep(10)

######################### Inicio da exportação do arquivo ".xls"
                        try:
                            # Aguarda a remoção do "blockUI"
                            wait.until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='blockUI blockOverlay']")))
                            # Aguarda o elemento ser clicável
                            wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='cphBody_ExportarBOLink']")))
                        
                        finally:
                            # Localiza o elemento de exportar e clica
                            print(strftime("Inicio da exportação em %d %b %Y às %H:%M:%S de: {} - {}/{}".format(caminho[11:], j, i), localtime())) #alterado
                            elemex = driver.find_element_by_xpath("//*[@id='cphBody_ExportarBOLink']")
                            elemex.click()
                            print(strftime("Finalização da exportação em %d %b %Y às %H:%M:%S de: {} - {}/{}".format(caminho[11:], j, i), localtime())) #alterado
                            sleep(30)




In [ ]:
# Finaliza o Webdriver
driver.quit()
print('Coleta finalizada')